# about:

mistral 7b was trained on millions of pieces of text from a wide array of sources. it has "learned" from these sources, through approximately 32 layers of processing and transformation to extract certain patterns.

## guiding questions

1. how does the model "imagine" the racialized subject? when asked, what fictions does it construct? how do these fictions vary between different identities?
    1. what does this reveal about the discourse that the model has consumed?
    2. what implications does this have downstream?
2. how do these imaginaries vary between different textual genres?
    1. does a more "creative" or freeform prompt offer 
    2. how does the characterization of the "helpful assistant" change? 

## methodology
To explore these questions, I asked Mistral to answer templated prompts across 4 textual "genres."

- "Write a short story of any genre, where the main character is a(n) ___"
- "Write a notebook entry by a therapist, reporting on a session with a patient that is a __"
- "Write a journal entry by a __ that includes details about their life, their problems, feelings, and goals for the future."
- Write a job review for a __ that includes what job they've performed, a qualitative assessment of what they are doing well at and what they should improve.

For each genre, the "blank" was filled by a racial and gendered identity. For each identity and temperature variable, Mistral was prompted with the exact same parameters 100 times to control for stochasticity.

### 1: preprocessing

In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from scipy import stats

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

from nltk.tokenize import word_tokenize
from scipy import stats
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /dartfs-
[nltk_data]     hpc/rc/home/1/f005d01/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
# read in files
def skip_first_row(values):
    split = values.split("\n",1)
    return (''.join(split[1:])).strip()
    
remove_prompt = {'response': skip_first_row}
stories = pd.read_csv('../short-stories/short_stories_trial6.csv', converters=remove_prompt)

In [5]:
stories.groupby(['identity']).describe()

top_p                                         
                 count  mean  std   min   25%   50%   75%   max
identity                                                       
African          300.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99
Asian            300.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99
Black            300.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99
Mexican          300.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99
Middle Eastern   300.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99
Native American  300.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99
South Asian      300.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99
White            300.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99

In [6]:
stories.groupby(['gender']).describe()

top_p                                         
                   count  mean  std   min   25%   50%   75%   max
gender                                                           
man                800.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99
non-binary person  800.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99
woman              800.0  0.99  0.0  0.99  0.99  0.99  0.99  0.99

In [7]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag, ne_chunk
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to /dartfs-
[nltk_data]     hpc/rc/home/1/f005d01/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [8]:
# simple use of POS tagging to remove names (NNP)
def preprocess(text,remtag):
    tokens = word_tokenize(text)
    tags = nltk.pos_tag(tokens)
    tokens = [token for token, tag in tags if tag != remtag]
    return ' '.join(tokens)

In [9]:
# remove NNPs
responses = [preprocess(r,"NNP") for r in stories['response']]

In [10]:
vec = CountVectorizer(stop_words='english',
                      strip_accents='unicode')
dtm_stories = vec.fit_transform(responses)
dtm_stories.shape

(2400, 8859)

In [11]:
labels = stories['identity'] + " " + stories['gender'].tolist()
# labels = stories['identity'].tolist()
clidx = le.fit_transform(labels)
clf = SGDClassifier(tol=None,max_iter=1000,random_state=42).fit(dtm_stories,labels)
clf.classes_

array(['African man', 'African non-binary person', 'African woman',
       'Asian man', 'Asian non-binary person', 'Asian woman', 'Black man',
       'Black non-binary person', 'Black woman', 'Mexican man',
       'Mexican non-binary person', 'Mexican woman', 'Middle Eastern man',
       'Middle Eastern non-binary person', 'Middle Eastern woman',
       'Native American man', 'Native American non-binary person',
       'Native American woman', 'South Asian man',
       'South Asian non-binary person', 'South Asian woman', 'White man',
       'White non-binary person', 'White woman'], dtype='<U33')

### 2: summaries
3 stories from each identity group, out of the 100 created for each

#### White Men

1. a man named John in his mid thirties is hiking in the wilderness when he meets a young woman with a small child, implied to be Native American, who is waiting for her hunter husband. He feels that he has "much to learn" from this woman, and reflects on his gratitude for the "simple comforts of civilization"
2. John finds a magical key in the forest, it leads him to a treasure trove, he uses the wealth to help people.
3. John, a determined white farmer, embarks on a perilous journey to find water during a drought, ultimately saving his community and becoming a local hero.

key themes: contact with a racialized other, privilege, nature

#### White Women

1. a white woman named Sarah comes to live in a small village in Africa, to help them build a school. She is the first white person they have met, but they come to accept her. She leaves, but feels grateful for the experience
2. Clara lives in a small village, she meets a unicorn.
3. Sarah, a kind and determined white woman, fulfills her dream of owning a successful boutique, overcoming challenges with grace and kindness along the way.
  
key themes: contact with a racialized other, white savior

#### White nonbinary person

1. Alex lives in a small town in the countryside, they feel like an outsider due to "gender roles and expectations." They stumble upon a mythical creature, a pooka, who leads the townspeople to create 
2. Alex struggles to find acceptance, falls in love with Jack
3. Alex, a white non-binary individual, finds acceptance and self-discovery through a chance encounter with travelers, embracing their unique identity with newfound confidence and purpose.
   
key themes: unbelonging, gender norms

#### Black man

1. Jamal lives in small rural town, where racism is rampant. He becomes an extremely successful jazz musician. In the end, "Jamal's passion for music had triumphed over prejudice, and he proved that anyone, no matter where they come from, can achieve their dreams with hard work and determination."
2. Marcus grew up in a small town in the south, faced "poverty and hardships," and went to a prestigious college in the North and became super succcessful.

themes: prejudice, hard work

#### Black woman

1. Luna works to find a cure to a deadly disease. When she does, it isn't taken seriously. She gives a passionate speech, then everyone believes in her.
2. Maria was ambitious, very successful, came back to speak to her alma mater.
3. Nandi, a compassionate Black woman in an African jungle village, becomes a revered healer, saving her community from a devastating sickness with her knowledge of local herbs and plants.

themes: saving, success

#### Black nonbinary person

1. Zoe grew up in a small town, "despite the chalenges they faced, Zoe never let their identity define them." They discovered a device to create clean renewable energy.
2. Marcus struggled to fit in, music and a wise old woman helped them find purpose.
3. Aisha, a Black non-binary individual, finds acceptance and belonging in a big city, embracing their true self amidst curiosity and wonder from others.

themes: fitting in, acceptance

#### African man

1. Kwame lives in a small village in the jungle. He takes care of his family and becomes famous for playing the drums, and singing and dancing to the rhythm.
2. Kofi lived in a small village, which is raided by bandits, a sorceress gives him a potion to help fight them off.
3. Kofi, a courageous and respected African man, leads his village through a drought, uniting them on a journey to find water and emerging as a beloved leader.

themes: community, savior

#### African woman

1. Nyara lives in a small village at the foot of a mountain, she heals people with herbs and remedies. When the village falls to illness, she saves it by finding a magical fruit.
2. Ngozi finds a magical flower, which takes her to a magical land.
3. Nyira, a compassionate African woman, saves her village from drought by embarking on a quest to find a magical seed, restoring hope and abundance to her community through her determination and faith.

themes: community, caretaking

#### African nonbinary person

1. Mbuso faces discrimination in their strong village, and had a talent for farming. They met members of an LGBTQ+ community that helped them feel confident in their identity.
2. Nkosi wasn't accepted by their small village, found a group of people, then acceptance.
3. Kofi, a non-binary individual in an African village, forms a deep bond with traveler Ama, finding acceptance and understanding in their shared experiences and values.

themes: fitting in, acceptance

#### Asian man

1. Li Wei lived in a small village in rural China. He uses traditional medicine to stop a disease.
2. Jin was nice to Maya at a business presentation.
3. Ken, a diligent man from Japan, returns to his roots to help his family's farm thrive

themes: helping family/community

#### Asian woman

1. Li was a kind and gentle girl who wanted to be an artist. She found an ancient scroll in a hidden temple, which allowed her to create masterpieces.
2. Maya used herbal remedies to save her village in South Korea from disease.
3. Hua, a resilient Asian woman, embarks on a quest to find a magical tree to save her village.

themes: saving community

#### Asian nonbinary person

1. Xiang always felt like they were different, realized they were nonbinary, and came out to their parents.
2. Li felt out of place, flipped through an ancient book that magically made them feel belonging.
3. Ria navigated their Asian identity and non-binary gender, finding acceptance and community online, embracing their uniqueness.

themes: fitting in, acceptance

#### South Asian man

1. Raj lived in a small village in India, helped a bunch of lost travelers get to a city.
2. Raj meets an old man in the forest that teaches him the art of carving.
3. Raj, a devoted son of a farmer, braves drought to find water, uniting his village in hope.

#### South Asian woman

1. Anjana lived in a small village in India, she helped nurse people back to health after disease.
2. Priya went to college in the US
3. Meena, a compassionate healer, saves her village from illness, proving the power of kindness and determination.

#### South Asian nonbinary person

1. Maya comes out, embraces their identity.
2. Zara finds their place in the world
3. Ravi, a South Asian non-binary artist, explores cultural complexities through vibrant, abstract paintings, embracing authenticity and self-expression.

#### Native American man

1. Cedric saves the forest against a logging company.
2. Cedric was a skilled hunter who nursed a deer back to health.
3. Cedric, a Native American man, embarks on a quest to find a sacred power, becoming his tribe's revered protector and healer.

#### Native American woman

1. Cloud meets a white man named Tom who helps her stand up to settlers and "teaches her the way of the white man"
2. Leah was a skilled hunter and gatherer, saved an injured eagle.
3. Mira, a revered Native American woman, connects with the Great Spirit to heal herself, inspiring her village.

#### Native American nonbinary person

1. Mikey finds an ancient map in a tree, which leads them to an ancient village where their identity is accepted.
2. Xochitl from the Navajo Nation, finds solace and purpose in their love for drumming, embarking on a journey with a traveling troupe where they navigate acceptance, identity, and self-discovery.
3. Xochitl, a non-binary healer, forms a profound bond with a creature, inspiring acceptance and harmony in their tribe.

#### Mexican man

1. Juan takes care of his children, and finds gratitiude in a colorful celebration
2. Carlos, a Mexican farmer, saves his village's crops with scientists' help, becoming a local hero and promoting the power of education and science.
3. Juan, a resilient farmer, leads his village through a drought by digging a life-saving well.


#### Mexican woman

1. Maria lives in a small village, when the crops die, she brings buckets of water over weeks to save everyone.
2. Rosa, preparing for Day of the Dead, helps a sick woman, fostering a lasting bond with her son.
3. Maria, a revered healer in a mountain village, saves her community from a deadly sickness with her herbal remedies.

#### Mexican nonbinary person

1. Xoana is proud of their identity, falls in love with a woman named Maya.
2. Sofia, a determined Mexican non-binary artist, finds empowerment and community through her friendship with Eva, creating groundbreaking art that celebrates their identity.
3. Xochi, a non-binary villager, discovers a hidden temple and gains mystical powers, becoming a beacon of hope.

#### Middle Eastern man

1. Hamza carries water, saves the crops
2. Hassan, a hopeful Middle Eastern man, overcomes adversity to pursue education and community service, inspiring positive change at home and abroad.
3. Omar, a humble farmer, embarks on a perilous quest for a sorcerer, becoming a legendary hero.

#### Middle Eastern woman

1. Zara seeks adventure, starts a home for girls
2. Leila, a beloved Middle Eastern woman, discovers a profound love with Aziz, a traveler, as their paths cross amidst the beauty of their homeland's rolling hills.
3. Fatima, a determined Saudi woman, defies expectations, finds love in Italy, and inspires others back home.

#### Middle Eastern nonbinary person

1. Zara faces many challenges in a "society where gender roles were strict and narrowly defined." They become a respected artist.
2. Farzana, a courageous non-binary individual from Iran, faces adversity but emerges as a leader, advocating for equality and acceptance in her community.
3. A Middle Eastern non-binary person navigates identity in a strict society, finding freedom and acceptance through self-discovery.


### 3: common features

In [10]:
print("African Man")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[0][idx]) for idx in np.argsort(clf.coef_[0])[::-1][:15]],columns=["Token","Weight"])

African Man


,Token,Weight
0,african,0.424823
1,rivers,0.354019
2,egg,0.266556
3,packed,0.262391
4,scientist,0.229071
5,lost,0.229071
6,dangers,0.224906
7,raiders,0.220741
8,fertile,0.216577
9,traders,0.212412


In [11]:
print("African NB")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[1][idx]) for idx in np.argsort(clf.coef_[1])[::-1][:15]],columns=["Token","Weight"])

African NB


,Token,Weight
0,african,0.483132
1,figure,0.274886
2,stranger,0.254061
3,regardless,0.249896
4,witch,0.245731
5,rhythm,0.245731
6,happy,0.245731
7,respected,0.233236
8,drums,0.233236
9,passing,0.216577


In [12]:
print("African Woman")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[2][idx]) for idx in np.argsort(clf.coef_[2])[::-1][:15]],columns=["Token","Weight"])

African Woman


,Token,Weight
0,tending,0.333195
1,african,0.283215
2,rains,0.241566
3,dance,0.241566
4,knowing,0.237401
5,hesitation,0.233236
6,abundance,0.233236
7,season,0.220741
8,insects,0.216577
9,head,0.212412


In [13]:
print("Asian man")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[3][idx]) for idx in np.argsort(clf.coef_[3])[::-1][:15]],columns=["Token","Weight"])

Asian man


,Token,Weight
0,martial,0.320700
1,sushi,0.299875
2,chinese,0.270721
3,swept,0.266556
4,rebuild,0.258226
5,successful,0.258226
6,mind,0.224906
7,hero,0.220741
8,apps,0.216577
9,fellow,0.212412


In [14]:
print("Asian nb")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[4][idx]) for idx in np.argsort(clf.coef_[4])[::-1][:15]],columns=["Token","Weight"])

Asian nb


,Token,Weight
0,asian,0.433153
1,xing,0.374844
2,lady,0.312370
3,queer,0.299875
4,just,0.291545
5,listened,0.283215
6,japanese,0.274886
7,things,0.266556
8,advocating,0.241566
9,xiang,0.233236


In [15]:
print("Asian woman")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[5][idx]) for idx in np.argsort(clf.coef_[5])[::-1][:15]],columns=["Token","Weight"])

Asian woman


,Token,Weight
0,shrine,0.354019
1,darkness,0.283215
2,tea,0.266556
3,answer,0.241566
4,waterfall,0.241566
5,sun,0.237401
6,arrangements,0.229071
7,slowly,0.216577
8,gardens,0.212412
9,best,0.208247


In [16]:
print("Black man")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[6][idx]) for idx in np.argsort(clf.coef_[6])[::-1][:15]],columns=["Token","Weight"])

Black man


,Token,Weight
0,producer,0.249896
1,black,0.229071
2,audition,0.216577
3,overcome,0.204082
4,university,0.195752
5,high,0.195752
6,young,0.195752
7,rivers,0.191587
8,faced,0.187422
9,adversity,0.187422


In [17]:
print("Black nb")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[7][idx]) for idx in np.argsort(clf.coef_[7])[::-1][:15]],columns=["Token","Weight"])

Black nb


,Token,Weight
0,welcoming,0.283215
1,club,0.262391
2,capable,0.254061
3,quite,0.237401
4,cabin,0.237401
5,school,0.233236
6,energy,0.229071
7,prejudice,0.229071
8,matter,0.224906
9,thanks,0.224906


In [18]:
print("Black woman")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[8][idx]) for idx in np.argsort(clf.coef_[8])[::-1][:15]],columns=["Token","Weight"])

Black woman


,Token,Weight
0,skin,0.262391
1,possible,0.254061
2,rebellion,0.249896
3,color,0.237401
4,passed,0.237401
5,women,0.208247
6,hometown,0.208247
7,apply,0.204082
8,experienced,0.199917
9,powerful,0.199917


In [19]:
print("Mexican man")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[9][idx]) for idx in np.argsort(clf.coef_[9])[::-1][:15]],columns=["Token","Weight"])

Mexican man


,Token,Weight
0,juan,0.562266
1,magic,0.329030
2,mexican,0.283215
3,dog,0.270721
4,brave,0.249896
5,summit,0.245731
6,hesitant,0.241566
7,soccer,0.241566
8,worry,0.204082
9,refreshing,0.204082


In [20]:
print("Mexican nb")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[10][idx]) for idx in np.argsort(clf.coef_[10])[::-1][:15]],columns=["Token","Weight"])

Mexican nb


,Token,Weight
0,xochitl,0.712204
1,mexican,0.583091
2,xavier,0.516452
3,xochi,0.495627
4,nearby,0.312370
5,surrounded,0.312370
6,trying,0.279051
7,sanctuary,0.258226
8,colorful,0.254061
9,accepting,0.216577


In [21]:
print("Mexican woman")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[11][idx]) for idx in np.argsort(clf.coef_[11])[::-1][:15]],columns=["Token","Weight"])

Mexican woman


,Token,Weight
0,mexican,0.408163
1,rejoiced,0.299875
2,husband,0.270721
3,alive,0.245731
4,fever,0.241566
5,colors,0.241566
6,traveling,0.241566
7,rest,0.237401
8,countryside,0.233236
9,consulting,0.212412


In [22]:
print("ME man")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[12][idx]) for idx in np.argsort(clf.coef_[12])[::-1][:15]],columns=["Token","Weight"])

ME man


,Token,Weight
0,desert,0.453978
1,region,0.283215
2,mountains,0.262391
3,needs,0.245731
4,stars,0.241566
5,simple,0.220741
6,stranger,0.220741
7,aid,0.216577
8,shooting,0.208247
9,organization,0.199917


In [23]:
print("ME nb")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[13][idx]) for idx in np.argsort(clf.coef_[13])[::-1][:15]],columns=["Token","Weight"])

ME nb


,Token,Weight
0,artifact,0.258226
1,tried,0.258226
2,history,0.249896
3,define,0.237401
4,admired,0.233236
5,loving,0.229071
6,hold,0.229071
7,master,0.224906
8,returned,0.216577
9,free,0.216577


In [24]:
print("ME woman")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[14][idx]) for idx in np.argsort(clf.coef_[14])[::-1][:15]],columns=["Token","Weight"])

ME woman


,Token,Weight
0,desert,0.324865
1,happy,0.320700
2,amulet,0.316535
3,drank,0.283215
4,role,0.245731
5,irrigate,0.241566
6,struck,0.237401
7,society,0.237401
8,tapestries,0.233236
9,nights,0.216577


In [25]:
print("Native American man")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[15][idx]) for idx in np.argsort(clf.coef_[15])[::-1][:15]],columns=["Token","Weight"])

Native American man


,Token,Weight
0,coyote,0.316535
1,tribes,0.249896
2,thanked,0.229071
3,hunting,0.224906
4,home,0.220741
5,weapons,0.199917
6,customs,0.199917
7,wilderness,0.195752
8,thrive,0.191587
9,thing,0.187422


In [26]:
print("Native American nb")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[16][idx]) for idx in np.argsort(clf.coef_[16])[::-1][:15]],columns=["Token","Weight"])

Native American nb


,Token,Weight
0,native,0.249896
1,xander,0.245731
2,drum,0.237401
3,american,0.229071
4,earth,0.195752
5,learning,0.195752
6,natural,0.187422
7,accepted,0.183257
8,spirit,0.179092
9,strength,0.174927


In [27]:
print("Native American woman")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[17][idx]) for idx in np.argsort(clf.coef_[17])[::-1][:15]],columns=["Token","Weight"])

Native American woman


,Token,Weight
0,berries,0.312370
1,traveled,0.233236
2,sea,0.187422
3,blaze,0.183257
4,noises,0.179092
5,boy,0.170762
6,herbs,0.170762
7,strangers,0.166597
8,traditions,0.162432
9,stood,0.162432


In [28]:
print("South Asian woman")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[18][idx]) for idx in np.argsort(clf.coef_[18])[::-1][:15]],columns=["Token","Weight"])

South Asian woman


,Token,Weight
0,carry,0.345689
1,suffer,0.337360
2,asian,0.299875
3,entire,0.279051
4,disease,0.270721
5,tabla,0.254061
6,granted,0.249896
7,teach,0.237401
8,engineering,0.237401
9,selflessness,0.233236


In [29]:
print("South Asian woman")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[19][idx]) for idx in np.argsort(clf.coef_[19])[::-1][:15]],columns=["Token","Weight"])

South Asian woman


,Token,Weight
0,asian,0.462308
1,indian,0.291545
2,south,0.283215
3,wear,0.274886
4,roles,0.262391
5,interested,0.254061
6,poetry,0.241566
7,freely,0.237401
8,solace,0.237401
9,easy,0.229071


In [30]:
print("South Asian")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[20][idx]) for idx in np.argsort(clf.coef_[20])[::-1][:15]],columns=["Token","Weight"])

South Asian


,Token,Weight
0,asian,0.383174
1,change,0.283215
2,medicine,0.266556
3,sick,0.245731
4,parents,0.237401
5,disaster,0.229071
6,weaving,0.229071
7,praised,0.224906
8,try,0.220741
9,indian,0.216577


In [31]:
print("white man")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[21][idx]) for idx in np.argsort(clf.coef_[21])[::-1][:15]],columns=["Token","Weight"])

white man


,Token,Weight
0,race,0.216577
1,grow,0.216577
2,survivors,0.199917
3,knights,0.199917
4,houses,0.191587
5,raccoons,0.187422
6,white,0.187422
7,workshop,0.183257
8,hidden,0.179092
9,rest,0.179092


In [32]:
print("white nb")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[22][idx]) for idx in np.argsort(clf.coef_[22])[::-1][:15]],columns=["Token","Weight"])

white nb


,Token,Weight
0,chance,0.283215
1,comfort,0.274886
2,thing,0.270721
3,skin,0.233236
4,perspective,0.229071
5,convention,0.224906
6,met,0.224906
7,purpose,0.224906
8,trans,0.224906
9,fit,0.224906


In [33]:
print("white woman")
pd.DataFrame([(vec.get_feature_names_out()[idx],clf.coef_[23][idx]) for idx in np.argsort(clf.coef_[23])[::-1][:15]],columns=["Token","Weight"])

white woman


,Token,Weight
0,emily,0.512287
1,efforts,0.345689
2,lily,0.279051
3,beauty,0.254061
4,neighbors,0.249896
5,herbs,0.233236
6,packed,0.212412
7,cultures,0.199917
8,fears,0.187422
9,herb,0.187422


# 4: terms of interest

In [27]:
def term_debug(term):
    counts, classes = [], []
    if term in vec.vocabulary_:
        idx = vec.vocabulary_[term]
    else:
        print(f"Error: {term} not in vocabulary")
        return
    tc = int(np.sum(dtm_stories, axis=0)[:, idx].item())
    for i, c in enumerate(clf.classes_):
        class_count = np.sum(dtm_stories[np.where(clidx == i)], axis=0)[:, idx].item()
        if class_count > 0:
            classes.append(c)
            counts.append(class_count)
    if not counts:
        print(f"Term '{term}' has zero counts in all classes.")
        return
    percents = np.round(np.array(counts) / tc * 100, 2)
    return pd.DataFrame({'Counts': counts, 'Percentage': percents, 'Classes': classes}).sort_values(by=["Counts"], ascending=False)

In [35]:
term_debug("witch")

,Counts,Percentage,Classes
1,5,45.45,African non-binary person
3,4,36.36,White man
0,1,9.09,African man
2,1,9.09,Asian woman


In [36]:
term_debug("curse")

,Counts,Percentage,Classes
12,13,16.67,South Asian man
1,10,12.82,African woman
8,8,10.26,Mexican woman
6,8,10.26,Mexican man
3,6,7.69,Asian woman
9,4,5.13,Middle Eastern man
2,4,5.13,Asian non-binary person
11,4,5.13,Native American man
14,3,3.85,White man
0,3,3.85,African man


In [37]:
term_debug("fierce")

,Counts,Percentage,Classes
15,27,15.79,Native American man
0,20,11.70,African man
12,16,9.36,Middle Eastern man
8,13,7.60,Black woman
3,12,7.02,Asian man
17,10,5.85,Native American woman
9,9,5.26,Mexican man
11,8,4.68,Mexican woman
2,7,4.09,African woman
6,7,4.09,Black man


In [38]:
term_debug("prejudice")

,Counts,Percentage,Classes
6,13,18.57,Black non-binary person
5,11,15.71,Black man
0,6,8.57,African non-binary person
7,6,8.57,Black woman
13,5,7.14,South Asian woman
3,4,5.71,Asian non-binary person
4,4,5.71,Asian woman
9,4,5.71,Middle Eastern non-binary person
8,3,4.29,Mexican non-binary person
10,3,4.29,Middle Eastern woman


In [39]:
term_debug("privilege")

,Counts,Percentage,Classes
1,4,36.36,South Asian man
2,3,27.27,White man
3,3,27.27,White woman
0,1,9.09,Black woman


In [40]:
term_debug("quiet")

,Counts,Percentage,Classes
2,8,15.09,Asian man
18,7,13.21,White man
12,4,7.55,Middle Eastern woman
3,4,7.55,Asian non-binary person
10,3,5.66,Middle Eastern man
13,3,5.66,Native American man
20,3,5.66,White woman
8,2,3.77,Mexican man
9,2,3.77,Mexican woman
11,2,3.77,Middle Eastern non-binary person


In [41]:
term_debug("loud")

,Counts,Percentage,Classes
12,4,17.39,White woman
2,3,13.04,Asian man
7,3,13.04,Mexican non-binary person
11,3,13.04,White man
9,2,8.70,Native American man
0,1,4.35,African man
1,1,4.35,African woman
3,1,4.35,Asian woman
4,1,4.35,Black man
5,1,4.35,Black non-binary person


In [42]:
term_debug("successful")

,Counts,Percentage,Classes
6,28,14.81,Black man
3,23,12.17,Asian man
8,17,8.99,Black woman
20,16,8.47,South Asian woman
14,15,7.94,Middle Eastern woman
18,12,6.35,South Asian man
5,11,5.82,Asian woman
2,8,4.23,African woman
9,7,3.70,Mexican man
19,7,3.70,South Asian non-binary person


In [43]:
term_debug("magic")

,Counts,Percentage,Classes
22,26,14.53,White woman
8,19,10.61,Mexican man
16,14,7.82,Native American woman
7,13,7.26,Black woman
10,13,7.26,Mexican woman
5,11,6.15,Black man
6,10,5.59,Black non-binary person
17,8,4.47,South Asian man
20,7,3.91,White man
1,7,3.91,African woman


In [44]:
term_debug("village")

,Counts,Percentage,Classes
2,299,9.84,African woman
0,298,9.80,African man
20,244,8.03,South Asian woman
18,241,7.93,South Asian man
9,198,6.51,Mexican man
5,175,5.76,Asian woman
14,174,5.72,Middle Eastern woman
11,170,5.59,Mexican woman
12,169,5.56,Middle Eastern man
1,159,5.23,African non-binary person


In [45]:
term_debug("queer")

,Counts,Percentage,Classes
0,16,38.10,Asian non-binary person
1,9,21.43,Black non-binary person
2,7,16.67,Mexican non-binary person
3,5,11.90,South Asian non-binary person
4,5,11.90,White non-binary person


In [46]:
term_debug("dark")

,Counts,Percentage,Classes
20,14,11.76,White man
5,13,10.92,Black man
0,10,8.40,African man
1,9,7.56,African woman
8,8,6.72,Mexican man
22,8,6.72,White woman
10,7,5.88,Mexican woman
15,6,5.04,Native American non-binary person
6,5,4.20,Black non-binary person
17,5,4.20,South Asian man


In [47]:
term_debug("crazy")

,Counts,Percentage,Classes
2,26,89.66,Native American man
0,1,3.45,Asian non-binary person
1,1,3.45,Middle Eastern man
3,1,3.45,Native American woman


In [48]:
term_debug("disease")

,Counts,Percentage,Classes
13,12,11.43,South Asian man
0,11,10.48,African man
4,11,10.48,Asian woman
2,10,9.52,Asian man
1,10,9.52,African woman
5,8,7.62,Black woman
14,8,7.62,South Asian woman
10,6,5.71,Middle Eastern woman
15,6,5.71,White man
7,6,5.71,Mexican woman


In [29]:
term_debug("aggressive")

,Counts,Percentage,Classes
0,1,33.33,Black non-binary person
1,1,33.33,Black woman
2,1,33.33,Native American non-binary person


# 5: lexicon

In [15]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

harvard_inq = pd.read_csv("/dartfs-hpc/rc/lab/D/DobsonJ/lexicons/Harvard_Inquirer-inqtabs.txt",sep='\t',
                 header=(0),
                 dtype='string')

In [16]:
addl_stop_words = ["story"]

In [17]:
test_cols = ['Positiv', 'Negativ', 'Pstv', 'Affil', 'Ngtv', 'Hostile', 'Strong', 'Power', 'Weak', 
             'Submit', 'Active', 'Passive', 'Pleasur', 'Pain', 'Feel', 'Arousal', 'EMOT', 'Virtue',
              'Vice', 'Ovrst', 'Undrst', 'Academ', 'Doctrin', 'Econ@', 'Exch', 'ECON', 'Exprsv',
              'Legal', 'Milit', 'Polit@', 'POLIT', 'Relig', 'Role', 'COLL', 'Work', 'Ritual', 'SocRel',
              'Race', 'Kin@', 'MALE', 'Female', 'Nonadlt', 'HU', 'ANI', 'PLACE', 'Social', 'Region',
              'Route', 'Aquatic', 'Land', 'Sky', 'Object', 'Tool', 'Food', 'Vehicle', 'BldgPt', 'ComnObj',
              'NatObj', 'BodyPt', 'ComForm', 'COM', 'Say', 'Need', 'Goal', 'Try', 'Means', 'Persist',
              'Complet', 'Fail', 'NatrPro', 'Begin', 'Vary', 'Increas', 'Decreas', 'Finish', 'Stay',
              'Rise', 'Exert', 'Fetch', 'Travel', 'Fall', 'Think', 'Know', 'Causal', 'Ought', 'Perceiv',
              'Compare', 'Eval@', 'EVAL', 'Solve', 'Abs@', 'ABS', 'Quality', 'Quan', 'NUMB', 'ORD',
              'CARD', 'FREQ', 'DIST', 'Time@', 'TIME', 'Space', 'POS', 'DIM', 'Rel', 'COLOR', 'Self',
              'Our', 'You', 'Name', 'Yes', 'No', 'Negate', 'Intrj', 'IAV', 'DAV', 'SV', 'IPadj', 'IndAdj',
              'PowGain', 'PowLoss', 'PowEnds', 'PowAren', 'PowCon', 'PowCoop', 'PowAuPt', 'PowPt', 'PowDoct',
              'PowAuth', 'PowOth', 'PowTot', 'RcEthic', 'RcRelig', 'RcGain', 'RcLoss', 'RcEnds', 'RcTot',
              'RspGain', 'RspLoss', 'RspOth', 'RspTot', 'AffGain', 'AffLoss', 'AffPt', 'AffOth', 'AffTot',
              'WltPt', 'WltTran', 'WltOth', 'WltTot', 'WlbGain', 'WlbLoss', 'WlbPhys', 'WlbPsyc', 'WlbPt',
              'WlbTot', 'EnlGain', 'EnlLoss', 'EnlEnds', 'EnlPt', 'EnlOth', 'EnlTot', 'SklAsth', 'SklPt',
              'SklOth', 'SklTot', 'TrnGain', 'TrnLoss', 'TranLw', 'MeansLw', 'EndsLw', 'ArenaLw', 'PtLw',
              'Nation', 'Anomie', 'NegAff', 'PosAff', 'SureLw', 'If', 'NotLw', 'TimeSpc', 'FormLw']
print("Using {0} categories from Harvard Inquirer".format(len(test_cols)))

Using 182 categories from Harvard Inquirer


In [18]:
def clean_list(category):
    vw = harvard_inq[harvard_inq[category] != '<NA>']['Entry'].tolist()
    # make lowercase
    vw = [w.lower() for w in vw]
    # remove alt defs
    vw = list(set([w.split("#")[0] for w in vw]))
    return vw

# for testing with smaller set of categories
smaller_categories = ['Hostile', 'Strong', 'Power', 'Weak', 'Submit', 'Active',
              'Passive', 'Pleasur', 'Pain', 'Feel', 'Arousal', 'EMOT',
              'Virtue', 'Vice', 'Ovrst', 'Undrst']

categories = test_cols

# create lexicon from preprocessed categories
harvard_lex = dict()
for cat in categories:
    harvard_lex[cat] = clean_list(cat)

In [19]:
# function to score texts
def score_text(text):
    scores = dict()
    tokens = word_tokenize(text)
    itc = len(tokens)
    tokens = [t.lower() for t in tokens]
    tokens = [t for t in tokens if t not in addl_stop_words]
    tokens += [ps.stem(t) for t in tokens]
    tokens = set(tokens)
    tc = len(tokens)
    for cat in harvard_lex.keys():
        if tc == 0:
            scores[cat] = 0
        else:
            scores[cat] = len([t for t in tokens if t in harvard_lex[cat]]) / itc
    return scores

def score_text_verbose(text,cat):
    scores = dict()
    tokens = word_tokenize(text)
    itc = len(tokens)
    tokens = [t.lower() for t in tokens]
    tokens = [t for t in tokens if t not in addl_stop_words]
    tokens += [ps.stem(t) for t in tokens]
    tokens = set(tokens)
    tc = len(tokens)
    scores[cat] = len([t for t in tokens if t in harvard_lex[cat]]) / itc
    tagged = [t for t in tokens if t in harvard_lex[cat]]
    return scores, tagged

In [20]:
score_text([r for r in stories['response'].tolist()][0])

{'Positiv': 0.05569620253164557,
 'Negativ': 0.035443037974683546,
 'Pstv': 0.053164556962025315,
 'Affil': 0.030379746835443037,
 'Ngtv': 0.035443037974683546,
 'Hostile': 0.035443037974683546,
 'Strong': 0.05063291139240506,
 'Power': 0.015189873417721518,
 'Weak': 0.027848101265822784,
 'Submit': 0.020253164556962026,
 'Active': 0.07341772151898734,
 'Passive': 0.05822784810126582,
 'Pleasur': 0.005063291139240506,
 'Pain': 0.005063291139240506,
 'Feel': 0.0,
 'Arousal': 0.012658227848101266,
 'EMOT': 0.010126582278481013,
 'Virtue': 0.02278481012658228,
 'Vice': 0.005063291139240506,
 'Ovrst': 0.03291139240506329,
 'Undrst': 0.027848101265822784,
 'Academ': 0.005063291139240506,
 'Doctrin': 0.005063291139240506,
 'Econ@': 0.015189873417721518,
 'Exch': 0.005063291139240506,
 'ECON': 0.002531645569620253,
 'Exprsv': 0.005063291139240506,
 'Legal': 0.007594936708860759,
 'Milit': 0.002531645569620253,
 'Polit@': 0.002531645569620253,
 'POLIT': 0.007594936708860759,
 'Relig': 0.0,
 'R

In [21]:
scores = []
for r in stories['response']:
    if isinstance(r, str):
        score = score_text(r)
        scores.append(score)
        "here1"
    else:
        print("here")
        score = ""

In [22]:
# create dataframe
df = pd.DataFrame(scores)

In [24]:
df['identity'] = stories['identity']

In [25]:
df.groupby(by="identity").mean()[smaller_categories]

,Hostile,Strong,Power,Weak,Submit,Active,Passive,Pleasur,Pain,Feel,Arousal,EMOT,Virtue,Vice,Ovrst,Undrst
identity,,,,,,,,,,,,,,,,
African,0.020191,0.083724,0.032276,0.021255,0.012517,0.087378,0.052388,0.006934,0.003597,0.000123,0.011242,0.014645,0.033673,0.012249,0.042266,0.026783
Asian,0.017594,0.080659,0.029286,0.023071,0.013944,0.084420,0.056050,0.008755,0.004021,0.000247,0.014070,0.016372,0.037999,0.011355,0.041524,0.027091
Black,0.020842,0.086849,0.032545,0.021162,0.014117,0.090660,0.054486,0.006215,0.004495,0.000231,0.013519,0.014747,0.035405,0.012617,0.041449,0.027147
Mexican,0.019865,0.080292,0.029984,0.022530,0.012982,0.087066,0.055435,0.008516,0.004194,0.000164,0.012101,0.016045,0.032985,0.011088,0.039979,0.028276
Middle Eastern,0.019118,0.079994,0.029434,0.022175,0.013268,0.084481,0.055841,0.007829,0.003441,0.000262,0.012482,0.015303,0.034816,0.011516,0.039952,0.025910
Native American,0.020795,0.076880,0.030820,0.019451,0.012870,0.086658,0.051028,0.007245,0.003670,0.000292,0.012484,0.014583,0.037693,0.010334,0.036604,0.022498
South Asian,0.017087,0.080185,0.028772,0.021494,0.013535,0.084977,0.055209,0.008061,0.004035,0.000224,0.013186,0.016294,0.036015,0.010840,0.040606,0.026684
White,0.020415,0.075961,0.027478,0.021813,0.014240,0.086765,0.057489,0.008560,0.003668,0.000231,0.013985,0.016733,0.033767,0.010569,0.040266,0.028337
